<a href="https://colab.research.google.com/github/kocairiserkan/ant-based-model/blob/main/Ant_Based_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mesa

In [ ]:
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import BaseScheduler
import numpy as np
import random
import matplotlib.pyplot as plt
import itertools

class Ant(Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.column = random.randint(0,self.model.grid.width-1) # Choose a random column to start in and set the row to 0
        self.row = 0
        self.pos = (self.column,self.row)
        self.add_pheromone() # Add pheromone to the current cell
        self.score = 0       # Initialize score to 0
        self.target = range(19,32) # Set the target range

    def find_neighbours(self):
        # Find the neighbouring cells for the current position
        self.neighbour = []
        if self.pos[0] == 0:
            self.neighbour.append((self.model.grid.width-1,self.pos[1]+1))
            self.neighbour.append((self.pos[0],self.pos[1]+1))
            self.neighbour.append((self.pos[0]+1,self.pos[1]+1))
        elif self.pos[0] == self.model.grid.width-1:
            self.neighbour.append((self.pos[0]-1,self.pos[1]+1))
            self.neighbour.append((self.pos[0],self.pos[1]+1))
            self.neighbour.append((0,self.pos[1]+1)) 
        else:
            self.neighbour.append((self.pos[0]-1,self.pos[1]+1))
            self.neighbour.append((self.pos[0],self.pos[1]+1))
            self.neighbour.append((self.pos[0]+1,self.pos[1]+1))
        return self.neighbour
    
    def add_pheromone(self):
        # Add pheromone to the current cell
        self.model.grid.the_grid[self.pos[1]][self.pos[0]] += 1

    def show_values_of_neighbours(self):
         # Show the values of pheromone for the neighbouring cells
        self.values = []
        for x,y in self.find_neighbours():
            self.values.append(self.model.grid.the_grid[y][x])
        return self.values
    
    def choose_neighbour(self):
        # Choose a neighbouring cell based on pheromone values
        result = all(neighbour == self.show_values_of_neighbours()[0] for neighbour in self.show_values_of_neighbours())
        if result:
            # If all neighbouring cells have the same pheromone level, choose randomly
            new_pos = random.choice(self.find_neighbours())
            return new_pos
        else:
            # Choose a cell based on probability distribution of pheromone levels
            total = sum(self.show_values_of_neighbours())+0.3
            self.prob_dist = [(x+0.1)/total for x in self.show_values_of_neighbours()]
            next_cell = random.choices(self.find_neighbours(), weights=self.prob_dist)[0]
            return next_cell
        


    def move(self):
        # Move the ant to the chosen neighbouring cell
        self.find_neighbours()
        next_pos = self.choose_neighbour()
        self.model.grid.move_agent(self, next_pos)
        self.add_pheromone()
        self.pos = next_pos
        
        # If the ant has reached the bottom row and is on the target range, update the score to 1
        if self.pos[1] == self.model.grid.height-1:
            if self.pos[0] in self.target: 
                self.score = 1


class Grid(SingleGrid):
    
    # Constructor that initializes the grid with a given width, height, and torus flag
    def __init__(self, width, height, torus):
        super().__init__(width, height, torus)
        self.width = width
        self.height = height
        self.the_grid = np.zeros((height, width)) # Initialize the grid with zeros
    
    # A method to show the current state of the grid
    def show_grid(self):
        return self.the_grid
    
    # A method to decay the pheromone level in the grid by a given factor
    def decay_pheromone(self, decay_factor):
        self.the_grid *= decay_factor
        
    # A method to subtract a given value from the pheromone level in the grid
    # If the resulting pheromone level is negative, set it to 0
    def subtract_d(self, d):
        self.the_grid -= d
        for i in range(self.the_grid.shape[0]):
            for j in range(self.the_grid.shape[1]):
                if self.the_grid[i][j] < 0:
                    self.the_grid[i][j] = 0
        

class AntModel(Model):
    
    # Constructor that initializes the model with a given number of ants
    def __init__(self, n):
        super().__init__()
        self.schedule = BaseScheduler(self) # Initialize the scheduler
        self.grid = Grid(width=50, height=500, torus=True)  # Initialize the grid
        self.n = n # Save the number of ants
        
        # Create and add the specified number of ants to the scheduler and the grid
        for i in range(self.n):
            ant = Ant(i,self)
            self.schedule.add(ant)
            self.grid.place_agent(ant,ant.pos)

    # A method to advance the model by one step
    def step(self):
        self.schedule.step()
        # Check if ant has reached the bottom
        for agent in self.schedule.agents:
            if agent.pos[1] == self.grid.height-1:
                self.running = False #If an ant has reached the bottom, stop the model
            else:
                agent.move() # Otherwise, move the ant
            self.total_score = agent.score # Update the total score of the model
        


# reward function to affect the decay
r = []
def reward(iteration, total_score, lambd):
    if iteration == 0:
        r.append(0)
    else:
        new_r = r[-1] + (total_score - r[-1])/lambd
        r.append(new_r)
    

#decay formula
def decay(r_i):
    return 1 - (1/(495*r_i+5))

# it runs the simulation and apply the necessary functions
def run_simulation(lamb, pheromone_decay, score):
    init_model = AntModel(1)
    while init_model.running:
        init_model.step()


    for i in range(15000):
        init_model.total_score = score
        reward(iteration=i,total_score=init_model.total_score,lambd = lamb)
        init_model.grid.decay_pheromone(decay(r[-1])) 
        init_model.grid.subtract_d(pheromone_decay)
        pheromone_vals = init_model.grid.show_grid()
        init_model = AntModel(1)
        init_model.grid.the_grid = pheromone_vals
        while init_model.running:
            init_model.step()

        #change range
        if i >= 5000:
            init_model.schedule.agents[0].target = range(0,13)
    
    #if the ant is in the range at the bottom of grid
    if np.argmax(init_model.grid.the_grid[-1]) in range(0,13):
        return 1
    else:
        return 0

In [ ]:
lamb = [1.5, 2, 2.2]
pheromone_decay = [0.01, 0.05, 0.1]
score = [0.5, 1, 1.5]

# Create list of all possible parameter combinations
param_combinations = list(itertools.product(lamb, pheromone_decay, score))

# Initialize variables to keep track of best parameter values and results
best_params = None
best_results = float('-inf')

# Loop through each parameter combination and run simulation
for params in param_combinations:
    lamb, pheromone_decay, score = params
    # Run simulation with current parameter values and record results
    results = run_simulation(lamb, pheromone_decay, score)
    # Check if current parameter values yield better results than previous best
    if results > best_results:
        best_params = params
        best_results = results

# Print the best parameter values and results
print("Best parameters:", best_params)
print("Best results:", best_results)

In [ ]:
init_model = AntModel(1)
while init_model.running:
    init_model.step()


for i in range(15000):
    init_model.total_score = 1
    reward(iteration=i,total_score=init_model.total_score,lambd=1.5)
    init_model.grid.decay_pheromone(decay(r[-1])) 
    init_model.grid.subtract_d(0.05)
    pheromone_vals = init_model.grid.show_grid() 
    init_model = AntModel(1)
    init_model.grid.the_grid = pheromone_vals
    while init_model.running:
        init_model.step()

    if i >= 5000:
        init_model.schedule.agents[0].target = range(0,13)

#Visualize
fig = plt.figure(figsize=(15, 15))
plt.imshow(pheromone_vals, interpolation="nearest")
plt.colorbar()
plt.show()